In [1]:
import random
import copy

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from Data_Load_Assign import *

In [2]:
# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("| using device:", device)

# Set random seeds
np.random.seed(0)
torch.manual_seed(0)
random.seed(0)

# Set B: batch_size
bsz = 10

| using device: cuda


In [3]:
# Load Dataset
train_data, test_data = load_MNIST()

# Get client dataloaders
iid_train_loader = iid_Assign(train_data, batch_size = bsz)
noniid_train_loader = non_iid_Assign(train_data, batch_size = bsz)
print("iid_train_loader length", len(iid_train_loader))
print("noniid_train_loader length", len(noniid_train_loader))

iid_train_loader length 100
noniid_train_loader length 100


In [4]:
# Test iid and non-iid is correct.

# iid
sample_iid = random.sample(iid_train_loader, 5)
for i in sample_iid:
    sample_label = torch.zeros(10)
    for (x,y) in i:
        sample_label += torch.sum(F.one_hot(y, num_classes=10), dim=0)
    print("iid: ", sample_label)

# non_iid
sample_noniid = random.sample(noniid_train_loader, 5)
for i in sample_noniid:
    sample_label = torch.zeros(10)
    for (x,y) in i:
        sample_label += torch.sum(F.one_hot(y, num_classes=10), dim=0)
    print("non-iid: ", sample_label)

# Q: what if a client get two subdatasets of the same number? like: 
# non-iid:  tensor([600.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])

iid:  tensor([59., 64., 71., 60., 70., 49., 64., 57., 61., 45.])
iid:  tensor([57., 66., 65., 63., 69., 47., 58., 53., 62., 60.])
iid:  tensor([56., 72., 72., 50., 60., 50., 56., 68., 52., 64.])
iid:  tensor([59., 60., 56., 57., 49., 40., 61., 65., 77., 76.])
iid:  tensor([64., 65., 51., 63., 71., 50., 67., 59., 67., 43.])
non-iid:  tensor([  0., 300.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 300.])
non-iid:  tensor([600.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])
non-iid:  tensor([300., 300.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])
non-iid:  tensor([300.,   0.,   0.,   0.,   0.,   0., 300.,   0.,   0.,   0.])
non-iid:  tensor([  0., 300.,   0.,   0.,   0.,   0.,   0.,   0., 300.,   0.])


In [5]:
# Model Defination

# Model 1: A simple multilayer-perceptron with 2-hidden
# layers with 200 units each using ReLu activations (199,210
# total parameters), which we refer to as the MNIST 2NN.

# define fully connected NN
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(784, 200);
        self.fc2 = nn.Linear(200, 200);
        self.out = nn.Linear(200, 10);

    def forward(self, x):
        x = x.flatten(1) # torch.Size([B,784])
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

print(MLP())
print("Total parameters of 2NN(MLP) is", num_params(MLP()))

# Model 2: A CNN with two 5x5 convolution layers (the first with
# 32 channels, the second with 64, each followed with 2x2
# max pooling), a fully connected layer with 512 units and
# ReLu activation, and a final softmax output layer (1,663,370
# total parameters).

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
    
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 4 * 4) # flatten
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        return x

print(CNN())
print("Total parameters of CNN is", num_params(CNN()))

# Q: The paper said there will be 1,663,370 total parameters. Something wrong?

MLP(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (out): Linear(in_features=200, out_features=10, bias=True)
)
Total parameters of 2NN(MLP) is 199210
CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)
Total parameters of CNN is 582026
